In [ ]:
! pip install jiwer
! pip install pybind11 fastwer
! pip install jamotools
! pip install g2pk
! pip install korean_romanizer

In [51]:
import jiwer

ground_truth = "hello world"
hypothesis = "hello duck"

wer = jiwer.wer(ground_truth, hypothesis)
mer = jiwer.mer(ground_truth, hypothesis)
wil = jiwer.wil(ground_truth, hypothesis)

# faster, because `compute_measures` only needs to perform the heavy lifting once:
measures = jiwer.compute_measures(ground_truth, hypothesis)
wer = measures['wer']
mer = measures['mer']
wil = measures['wil']

In [92]:
import os

path=  '/content/drive/MyDrive/Voice_Data/KoG2P/txt_dataframe/concat/' 
os.listdir(path)

['training_all_text.csv',
 'training_all_utt2spk.csv',
 'training_all_spk2utt.csv',
 'training_all_null_path.csv',
 'val_all_text.csv',
 'val_all_utt2spk.csv',
 'val_all_spk2utt.csv',
 'val_all_null_path.csv',
 'training_all_spk2gender.csv',
 'val_all_spk2gender.csv',
 'concat_txt.ipynb']

In [127]:
import pandas as pd
import numpy as np


txt = pd.read_csv(path+'training_all_text.csv')
ord = pd.read_csv('/content/drive/MyDrive/Voice_Data/KoG2P/txt_dataframe/all_df/clean_txt_final.csv')

In [144]:
import fastwer
import jamotools
import jiwer
from korean_romanizer.romanizer import Romanizer

def CER(ground_truth, users):

    transformation = jiwer.Compose([
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemoveWhiteSpace(replace_by_space=False),
    jiwer.RemovePunctuation()
    ]) 

    ground_truth_phoneme = ground_truth
    hypothesis_phoneme = users

    ground_truth_char = jamotools.join_jamos(ground_truth_phoneme)
    hypothesis_char = jamotools.join_jamos(hypothesis_phoneme)

    wer = jiwer.wer(ground_truth_char, hypothesis_char) ## Word Error Rate
    mer = jiwer.mer(ground_truth_char, hypothesis_char) ## Match Error Rate
    wil = jiwer.wil(ground_truth_char, hypothesis_char) ## Word Information Lost

    CER_PHONEME = fastwer.score(list(transformation(ground_truth_phoneme)), list(transformation(hypothesis_phoneme)), char_level=True) ## Phoneme Error Rate
    CER_CHAR = fastwer.score(list(transformation(ground_truth_char)), list(transformation(hypothesis_char)), char_level=True) ## Char Error Rate

    print(len(ground_truth_phoneme), len(hypothesis_phoneme))
    print(f"ground_truth(phoneme): {ground_truth_phoneme}")
    print(f"hypothesis(phoneme): {hypothesis_phoneme}")

    print('='*30)
    print(len(ground_truth_char), len(hypothesis_char))
    print(f"ground_truth(char): {ground_truth_char}")
    print(f"hypothesis(char): {hypothesis_char}")


    print('='*30)
    print(f"wer:{wer:.4f}, mer:{mer:.4f}, wil:{wil:.4f}") 
    print(f"CER(char): {CER_CHAR}")
    print(f"CER(phoneme): {CER_PHONEME}")

    if CER_PHONEME==0:
        print("Perfect")
    elif 0<CER_PHONEME<=2:
        print('Accuracy: 98~99%')
    elif 2<CER_PHONEME<=10:
        print('Accuracy: 90~95%')
    elif CER_PHONEME>10:
        print('Accuracy: Below 90%')

    error_part = [{"idx":i, 'ground_truth':k, 'Users': t} for i,k in enumerate(list(ground_truth_char)) for j,t in enumerate(list(hypothesis_char)) if i==j if k != t]
    idx = [i for i,k in enumerate(list(ground_truth_char)) for j,t in enumerate(list(hypothesis_char)) if i==j if k != t]
    true = [k for i,k in enumerate(list(ground_truth_char)) for j,t in enumerate(list(hypothesis_char)) if i==j if k != t]
    false = [t for i,k in enumerate(list(ground_truth_char)) for j,t in enumerate(list(hypothesis_char)) if i==j if k != t]

    print('='*30)
    print(f"error_idx:{idx}")
    print("Modify")
    print("ordinal ==> change")
    print()

    for i,k in enumerate(idx):

        true_roma = Romanizer("{}".format(ground_truth_char[k-1:k+4]))    
        user_roma = Romanizer("{}".format(hypothesis_char[k-1:k+4]))


        print(f"{hypothesis_char[k-1:k+4]}({user_roma.romanize()}) ==> {ground_truth_char[k-1:k+4]}({true_roma.romanize()})")

    print("="*30)
    print('Final')
    print()

    print('Your Pronounce: ',Romanizer("{}".format(hypothesis_char)).romanize())
    print('Right Pronounce: ',Romanizer("{}".format(ground_truth_char)).romanize())

        



In [150]:
a = np.random.randint(ord.shape[0])
ground_truth = txt.txt[a]
users = jamotools.split_syllables(u"{}".format(ord.txt[a]))

CER(ground_truth, users)

41 42
ground_truth(phoneme): ㄱㅡ ㄴㅏㄴㅡㄴ ㅋㅗㄹㄹㅏ ㅁㅏㄹㄱㅗ ㅍㅏㅂㅋㅗㄴㅁㅏㄴ ㅁㅓㄱㄲㅗ ㅅㅣㅍㅓ
hypothesis(phoneme): ㄱㅡ ㄴㅏㄴㅡㄴ ㅋㅗㄹㄹㅏ ㅁㅏㄹㄱㅗ ㅍㅏㅂㅋㅗㄴㅁㅏㄴ ㅁㅓㄱㄱㅗ ㅅㅣㅍㅇㅓ
20 20
ground_truth(char): 그 나는 콜라 말고 팝콘만 먹꼬 시퍼
hypothesis(char): 그 나는 콜라 말고 팝콘만 먹고 싶어
wer:0.2857, mer:0.2857, wil:0.4898
CER(char): 14.2857
CER(phoneme): 1.9048
Accuracy: 98~99%
error_idx:[16, 18, 19]
Modify
ordinal ==> change

먹고 싶어(meokgo sibeo) ==> 먹꼬 시퍼(meokkko sipeo)
 싶어( sibeo) ==>  시퍼( sipeo)
싶어(sibeo) ==> 시퍼(sipeo)
Final

Your Pronounce:  geu naneun kolra malgo papkonman meokgo sibeo
Right Pronounce:  geu naneun kolra malgo papkonman meokkko sipeo
